In [ ]:
# !wget https://www.dropbox.com/s/w8kiecmkgzemewv/data.zip?dl=0

--2023-03-08 13:38:27--  https://www.dropbox.com/s/w8kiecmkgzemewv/data.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/w8kiecmkgzemewv/data.zip [following]
--2023-03-08 13:38:28--  https://www.dropbox.com/s/raw/w8kiecmkgzemewv/data.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucaf27e404aedc7055fba082f45a.dl.dropboxusercontent.com/cd/0/inline/B30LVpKo8xV4mOH4kp-swv1xWkCyUUi5FNRLtGZ0TOJYew_S9fd7oCJnkmAkvsUnsXK5O2F7H63xo7OMuRQhZXqq8XzXsDE4Qdsw2Xkv19_0gx1JDSSYm7FivETzlv2FGiQ-jrCPhZn-y_UjZu-_az_8VwKFHFs1C7Qmk5-GzukVlQ/file# [following]
--2023-03-08 13:38:28--  https://ucaf27e404aedc7055fba082f45a.dl.dropboxusercontent.com/cd/0/inline/B30LVpKo8xV4mOH4kp-swv1xWkCyUUi5FNRLtGZ0TOJYew_S9fd7oCJnkmAkvsUnsXK5O2F7H63xo7OMuRQh

In [ ]:
# !unzip -q "/content/data.zip?dl=0"

In [ ]:
!wget https://www.dropbox.com/s/lqe9rvhm036c4lu/ml-latest-small.zip?dl=0

--2023-03-08 15:58:04--  https://www.dropbox.com/s/lqe9rvhm036c4lu/ml-latest-small.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/lqe9rvhm036c4lu/ml-latest-small.zip [following]
--2023-03-08 15:58:05--  https://www.dropbox.com/s/raw/lqe9rvhm036c4lu/ml-latest-small.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf00cb99ade5fb10b73a055e450.dl.dropboxusercontent.com/cd/0/inline/B32UaEcNQR-ejuZdwYEumlfQLDz4D28a48_83sqZLDNQptAw6OZOXRcoJBE3F2tkfjfHmFZSPvdT7vj8H2CkRG0HPpXAU2W1GHdsr4MCS5uhOUlEGSMjjqIbFDchwSxYBmF6qnndvmQsgq27OaBMdSByxQTR2Dhb4pBDDgpHWwBfGg/file# [following]
--2023-03-08 15:58:05--  https://ucf00cb99ade5fb10b73a055e450.dl.dropboxusercontent.com/cd/0/inline/B32UaEcNQR-ejuZdwYEumlfQLDz4D28a48_83sqZLDNQptAw6OZOXR

In [ ]:
!unzip -q "/content/ml-latest-small.zip?dl=0"

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 15.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=765aae30b9d29422f66c61074029bab4b3da3d912e2bab9a155535711f490007
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create a SparkSession
spark = SparkSession.builder.appName('MovieRecommendation').getOrCreate()

# Load the ratings dataset and cache it
ratings = spark.read.format('csv').options(header='true', inferSchema='true').load('/content/ml-latest-small/ratings.csv').cache()

# Load the movies dataset and cache it
movies = spark.read.format('csv').options(header='true', inferSchema='true').load('/content/ml-latest-small/movies.csv').cache()

# Split the ratings dataset into training, validation, and test datasets
(training, validation, test) = ratings.randomSplit([0.6, 0.2, 0.2], seed=42)

# Define the ALS model
als = ALS(userCol='userId', itemCol='movieId', ratingCol='rating', nonnegative=True, coldStartStrategy='drop')

# Define the parameter grid for tuning
param_grid = ParamGridBuilder() \
    .addGrid(als.rank, [50, 100]) \
    .addGrid(als.regParam, [0.1, 0.05, 0.01]) \
    .build()

# Define the evaluation metric
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

# Define the cross validator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

# Train the ALS model using the training dataset
best_model = cv.fit(training)

# Use the validation dataset to select the best model
predictions = best_model.transform(validation)
rmse = evaluator.evaluate(predictions)
print('Root-mean-square error =', rmse)

# Use the test dataset to evaluate the performance of the best model
predictions = best_model.transform(test)
rmse = evaluator.evaluate(predictions)
print('Root-mean-square error on test set =', rmse)

# Save the best model for future use
best_model.bestModel.write().overwrite().save('/content/ALS_model.parquet')

# Load the saved model
saved_model = ALSModel.load('/content/ALS_model.parquet')

# Define the user ID for which we want to recommend movies
user_id = 1

# Get the movies the user has rated
user_ratings = ratings.filter(col('userId') == user_id)

# Print the movies the user likes and dislikes
print('Movies liked by user:')
user_ratings.filter(col('rating') >= 4).join(movies, 'movieId').select('title').show(truncate=False)
print('Movies disliked by user:')
user_ratings.filter(col('rating') < 4).join(movies, 'movieId').select('title').show(truncate=False)


Root-mean-square error = 0.9006727378630561
Root-mean-square error on test set = 0.8954534587525965
Movies liked by user:
+-----------------------------------------+
|title                                    |
+-----------------------------------------+
|Toy Story (1995)                         |
|Grumpier Old Men (1995)                  |
|Heat (1995)                              |
|Seven (a.k.a. Se7en) (1995)              |
|Usual Suspects, The (1995)               |
|Bottle Rocket (1996)                     |
|Braveheart (1995)                        |
|Rob Roy (1995)                           |
|Canadian Bacon (1995)                    |
|Desperado (1995)                         |
|Billy Madison (1995)                     |
|Dumb & Dumber (Dumb and Dumber) (1994)   |
|Ed Wood (1994)                           |
|Star Wars: Episode IV - A New Hope (1977)|
|Tommy Boy (1995)                         |
|Clear and Present Danger (1994)          |
|Forrest Gump (1994)                      

In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create a SparkSession
spark = SparkSession.builder.appName('MovieRecommendation').getOrCreate()

# Load the ratings dataset and cache it
ratings = spark.read.format('csv').options(header='true', inferSchema='true').load('/content/ml-latest-small/ratings.csv').cache()

# Load the movies dataset and cache it
movies = spark.read.format('csv').options(header='true', inferSchema='true').load('/content/ml-latest-small/movies.csv').cache()

# Split the ratings dataset into training, validation, and test datasets
(training, validation, test) = ratings.randomSplit([0.6, 0.2, 0.2], seed=42)

# Define the ALS model
als = ALS(userCol='userId', itemCol='movieId', ratingCol='rating', nonnegative=True, coldStartStrategy='drop')

# Define the parameter grid for tuning
param_grid = ParamGridBuilder() \
    .addGrid(als.rank, [10, 50]) \
    .addGrid(als.regParam, [0.1, 0.01]) \
    .build()

# Define the evaluation metric
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

# Define the cross validator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

# Train the ALS model using the training dataset
best_model = cv.fit(training)

# Use the validation dataset to select the best model
predictions = best_model.transform(validation)
rmse = evaluator.evaluate(predictions)
print('Root-mean-square error =', rmse)

# Use the test dataset to evaluate the performance of the best model
predictions = best_model.transform(test)
rmse = evaluator.evaluate(predictions)
print('Root-mean-square error on test set =', rmse)

# Save the best model for future use
best_model.bestModel.write().overwrite().save('/content/ALS_model.parquet')

# Load the saved model
saved_model = ALSModel.load('/content/ALS_model.parquet')

# Define the user ID for which we want to recommend movies
user_id = 1

# Get the movies the user has rated
user_ratings = ratings.filter(col('userId') == user_id)

# Print the movies the user likes and dislikes
print('Movies liked by user:')
user_ratings.filter(col('rating') >= 4).join(movies, 'movieId').select('title').show(truncate=False)
print('Movies disliked by user:')
user_ratings.filter(col('rating') < 4).join(movies, 'movieId').select('title').show(truncate=False)

Root-mean-square error = 0.9019296974533724
Root-mean-square error on test set = 0.8964187193865248
Movies liked by user:
+-----------------------------------------+
|title                                    |
+-----------------------------------------+
|Toy Story (1995)                         |
|Grumpier Old Men (1995)                  |
|Heat (1995)                              |
|Seven (a.k.a. Se7en) (1995)              |
|Usual Suspects, The (1995)               |
|Bottle Rocket (1996)                     |
|Braveheart (1995)                        |
|Rob Roy (1995)                           |
|Canadian Bacon (1995)                    |
|Desperado (1995)                         |
|Billy Madison (1995)                     |
|Dumb & Dumber (Dumb and Dumber) (1994)   |
|Ed Wood (1994)                           |
|Star Wars: Episode IV - A New Hope (1977)|
|Tommy Boy (1995)                         |
|Clear and Present Danger (1994)          |
|Forrest Gump (1994)                      

In [ ]:
!zip -r /content/ALSmodel.zip /content/ALS_model.parquet

  adding: content/ALS_model.parquet/ (stored 0%)
  adding: content/ALS_model.parquet/itemFactors/ (stored 0%)
  adding: content/ALS_model.parquet/itemFactors/.part-00005-562c9d5a-024a-4838-b283-be48d4848883-c000.snappy.parquet.crc (stored 0%)
  adding: content/ALS_model.parquet/itemFactors/._SUCCESS.crc (stored 0%)
  adding: content/ALS_model.parquet/itemFactors/part-00005-562c9d5a-024a-4838-b283-be48d4848883-c000.snappy.parquet (deflated 13%)
  adding: content/ALS_model.parquet/itemFactors/.part-00001-562c9d5a-024a-4838-b283-be48d4848883-c000.snappy.parquet.crc (stored 0%)
  adding: content/ALS_model.parquet/itemFactors/.part-00003-562c9d5a-024a-4838-b283-be48d4848883-c000.snappy.parquet.crc (stored 0%)
  adding: content/ALS_model.parquet/itemFactors/.part-00002-562c9d5a-024a-4838-b283-be48d4848883-c000.snappy.parquet.crc (stored 0%)
  adding: content/ALS_model.parquet/itemFactors/part-00000-562c9d5a-024a-4838-b283-be48d4848883-c000.snappy.parquet (deflated 11%)
  adding: content/ALS_

In [ ]:
from google.colab import files
files.download("/content/ALSmodel.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Get the top 10 recommended movies for the user
user_recs = saved_model.recommendForAllUsers(10).filter(col('userId') == user_id)

# Extract the recommended movie IDs and explode them into separate rows
user_recs = user_recs.selectExpr("explode(recommendations) as rec")
user_recs = user_recs.select('rec.movieId')

# Join with the movies DataFrame to get the movie titles
user_recs = user_recs.join(movies, 'movieId')

# Print the top 10 recommended movies for the user
print('Top 10 recommended movies for the user:')
user_recs.select('title').show(truncate=False)

Top 10 recommended movies for the user:
+------------------------------------------------+
|title                                           |
+------------------------------------------------+
|Doctor Zhivago (1965)                           |
|Three Billboards Outside Ebbing, Missouri (2017)|
|Harold and Maude (1971)                         |
|Yojimbo (1961)                                  |
|High Noon (1952)                                |
|Waiting for Guffman (1996)                      |
|Cool Hand Luke (1967)                           |
|Star Wars: Episode IV - A New Hope (1977)       |
|Cosmos                                          |
|It's a Wonderful Life (1946)                    |
+------------------------------------------------+



Root-mean-square error = 0.8996581610583839
Root-mean-square error on test set = 0.8965793681020697


IllegalArgumentException: ignored